In [1]:
from loader import Loader
L = Loader()

In [2]:
L.load_case_verdicts()

In [3]:
L.case_verdict_df.shape

(20, 3)

In [5]:
L.case_verdict_df['c'] = L.case_verdict_df['case_text'].str.count('mapa')

In [6]:
L.case_verdict_df[L.case_verdict_df['c'] > 0]

,date,case_id,case_text,c
